In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import time 
#from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import plotly.graph_objects as go
import re
# Natural Language Tool Kit 
import nltk  
nltk.download('stopwords') 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
submission =  pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

missing = train.isnull() 
sns.heatmap(missing, yticklabels=False)

sns.countplot(x=train['target'], data=train)


#test['text']=test['text'].apply(lambda x : remove_html(x))


def clean_text(text):
 
    text = re.sub('[^a-zA-Z]', ' ', text)  

    text = text.lower()  

    # split to array(default delimiter is " ") 
    text = text.split()  
    
    text = [w for w in text if not w in set(stopwords.words('english'))] 

    text = ' '.join(text)    
            
    return text


train['text'] = train['text'].apply(lambda x : clean_text(x))

test['text']=test['text'].apply(lambda x : clean_text(x))


train_lenth=6612

# The maximum number of words to be used. (most frequent)
train_sen=train.text[0:train_lenth]
train_label=train.target[0:train_lenth]

test_sen=train.text[train_lenth:]
test_label=train.target[train_lenth:]

########


tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(train_sen)

word_index = tokenizer.word_index


train_seq = tokenizer.texts_to_sequences(train_sen)
train_pad= keras.preprocessing.sequence.pad_sequences(train_seq, padding='post', maxlen=20, value=0)


test_seq = tokenizer.texts_to_sequences(test_sen)
test_pad= keras.preprocessing.sequence.pad_sequences(test_seq, padding='post', maxlen=20, value=0)

###  reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


# Model Definition with LSTM

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(50000, 32, input_length=20),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(14, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # remember this is a binary clasification
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


start=time.time()

model.fit(train_pad, train_label, epochs=10, validation_data=(test_pad, test_label))

final=(time.time()- start)/60

print("Time taken : {}".format(final))


predictions = model.predict_classes(test_pad)

In [ ]:

###  WORK WITH TEST DATASET NOW''''''